In [ ]:
from infrastructure.classes import Experiment, TrainParameters
from infrastructure.training import train
import torch

experiment = Experiment(
    tensor_parameters=dict(dtype=torch.float32, device=torch.device("cpu")),
    dataset_name="fnlvqr_glasses",
    dataset_number_of_points=10**6,
    dataloader_parameters=dict(batch_size=256, shuffle=True),
    pushforward_operator_name="amortized_neural_quantile_regression",
    pushforward_operator_parameters=dict(
        feature_dimension=1,
        response_dimension=1,
        hidden_dimension=64,
        number_of_hidden_layers=8,
        activation_function_name="Softplus",
        network_type="PISCNN",
        potential_to_estimate_with_neural_network="u"
    ),
    train_parameters=TrainParameters(
        number_of_epochs_to_train=100,
        verbose=True,
        optimizer_parameters=dict(lr=0.01),
        scheduler_parameters=dict(eta_min=0)
    )
)

model = train(experiment)
_ = model.eval()

Epoch: 8, Potential Objective: 0.750, Amortization Objective: 0.486, LR: 0.000158:  70%|███████   | 7/10 [1:07:16<27:58, 559.48s/it]

In [2]:
%matplotlib inline
from utils.plot import plot_quantile_levels_from_dataset
from datasets import PICNN_FNLVQR_Glasses
dataset = PICNN_FNLVQR_Glasses(tensor_parameters=experiment.tensor_parameters)

plot_quantile_levels_from_dataset(
    model=model,
    dataset=dataset,
    conditional_value=torch.Tensor([[.9]]),
    number_of_quantile_levels=10,
    tensor_parameters=experiment.tensor_parameters
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x2 and 1x8)